# Creando el grafo de leyes

Cargamos los datos que estan guardados en un json

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
import json

with open("data/parsed_law.json", "r", encoding="utf-8") as f:
    law_chile = json.load(f)

for law_code in law_chile:
    law = law_chile[law_code]
    number = law["numero"]
    law["public_name"] = f"Ley {number}"

Definimos la función que crea el grafo

In [7]:
def build_law_graph(laws):
    G = nx.DiGraph()

    for law_code in laws:
        law = laws[law_code]
        number = law["numero"]
        organism = law["organismo"]
        public_name = law["public_name"]
        parrafos = law["parrafos"]
        # print(f"Ley {number}, con codigo {law_code}, del organismo {organism}")

        if number.isnumeric():
            pass
        else:
            # si no es numerico continuamos
            # osea lo omitimos
            continue

        
        texto = ""
        for parr in parrafos:

            texto += parr["texto"]
            
        # print(f"ley {number}, con codigo {law_code} {texto[:100]}")
        G.add_node(
            law_code,
            tipo="Ley",
            numero=number,
            texto = texto,
            organismo=organism,
            publlic_name=public_name,
        )
        

        # Esto es para los parrafos, basicamente tener la ley apuntando a sus parrafos
        # no estoy seguro si lo usare.

        # for i, p in enumerate(law.get("parrafos", [])):
        #     p_id = p.get("id", f"{law_code}_p{i}")
        #     G.add_node(
        #         p_id,
        #         tipo=p.get("tipo", "Parrafo"),
        #         texto=p.get("texto", ""),
        #         derogado=p.get("derogado", "no derogado"),
        #         organismo=organism,  # mismo organismo que la ley
        #     )
        #     G.add_edge(law_code, p_id, tipo="contiene")

        for target in law.get("modified_to", []):
            
            G.add_edge(law_code, str(target), tipo="modifica")

    return G




In [8]:
G = build_law_graph(law_chile)

In [9]:
print(G)

DiGraph with 15462 nodes and 4593 edges


# Exportar grafo

Lo exportamos para visualizarlo en cytoscape


In [ ]:
import lxml

nx.write_graphml(G, "graph_law.graphml")

# Guardar el grafo para reutilizarlo

Basicamente usare pickle para guardar y cargar el grafo.


In [53]:
# Source - https://stackoverflow.com/questions/64846222/store-networkx-graph-object
# Posted by PatrickT
# Retrieved 5/11/2025, License - CC-BY-SA 4.0

import pickle

# save graph object to file
pickle.dump(G, open('graph_law.pickle', 'wb'))


# load graph object from file
G = pickle.load(open('graph_law.pickle', 'rb'))

In [55]:
print(G)

DiGraph with 15462 nodes and 4593 edges
